In [52]:
import networkx as nx
import numpy as np
import itertools
from scipy.spatial import distance
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)
import os
import sys
import random


new_path=os.getcwd()
new_path =new_path[:-13]
if new_path not in sys.path:
    sys.path.append(new_path)

#
import dmk_cont
import pre_extraction
#

In this tutorial we will show you how to use the ```pre_extr``` function contained in the ```pre_extraction``` script.

# PlotlyVisualizationModule 1: *Barycenters and Grid*.

## The Grid

In [5]:
ndiv = 10

In [10]:
grid, subgrid, points, vertices, coord,topol,element_attributes = dmk_cont.grid_gen(ndiv)

319


In [53]:
triang_source_indices = [0]#<-- not important for the tutorial
triang_sink_indices = [1]#<-- same here
weights = np.zeros(len(topol))

for i in range(len(topol)):
    weights[i] = random.uniform(0, 1) # defining random weights for the triangles

G_bar,G_triang,dict_seq = pre_extraction.dmk2pre_extr_inputs(coord, 
                                                             topol, 
                                                             weights,
                                                             triang_source_indices,
                                                             triang_sink_indices)

Gbar 319


In [54]:
node_trace_triang = go.Scatter(
    x=[],
    y=[],
    text=[],#enum_verteG_triang,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        color='red',
        #showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        #reversescale=True,
        #color='skyblue',
        size=10,
        line=dict(width=2)))

for node in G_triang.nodes():
    x, y = G_triang.nodes[node]['pos']
    node_trace_triang['x'] += tuple([x])
    node_trace_triang['y'] += tuple([y])
    

edge_trace_triang = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G_triang.edges():
    x0, y0 = G_triang.nodes[edge[0]]['pos']
    x1, y1 = G_triang.nodes[edge[1]]['pos']
    edge_trace_triang['x'] += tuple([x0, x1, None])
    edge_trace_triang['y'] += tuple([y0, y1, None])

fig5 = go.Figure(data=[edge_trace_triang, node_trace_triang],
                 layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                title='<br>',
                titlefont=dict(size=16),
                showlegend=True,
                hovermode='closest',
                autosize=False,
                width=900,
                height=800,
                margin=dict(b=20,l=10,r=5,t=40),
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))
                 
iplot(fig5, filename='networkx')

## The Barycenters 

In [55]:
node_trace_bar_weights= go.Scatter(
    x=[],
    y=[],
    text=weights,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color=weights,#<---------------------- new!
        size=20,
        colorbar=dict(
            thickness=5,
            title='Weights (tdens)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G_bar.nodes():
    x, y = G_bar.nodes[node]['pos']
    node_trace_bar_weights['x'] += tuple([x])
    node_trace_bar_weights['y'] += tuple([y])

    
fig2 = go.Figure(data=[node_trace_bar_weights,
                       edge_trace_triang, node_trace_triang
                       ],
             layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                title='<br>Positions of the barycenters w color given by weight',
                titlefont=dict(size=16),
                showlegend=True,
                hovermode='closest',
                 autosize=False,
                width=900,
                height=900,
                margin=dict(b=20,l=20,r=20,t=20),
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))

iplot(fig2, filename='networkx')

# PlotlyVisualizationModule 2: *Neighbors of a triangle*.

In [56]:
Node_=30

nodes = pre_extraction.get_first_neig(Node_, dict_seq)

dict_sec_neig, index_ = pre_extraction.get_sec_neig(Node_, dict_seq)

In [58]:
neig_triang={}
for triang in dict_sec_neig[Node_]:
    triang=list(triang)
    #print(triang)
    for n in triang:
        #print(n)
        x2,y2= G_triang.nodes[n]['pos']
        #print(x2,y2)
        neig_triang[n]=dict(
            x=x2,
            y=y2,
            xref='x',
            yref='y',
            text='ver',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=0,
            bgcolor='#ff7f0e')

for node in G_triang.nodes():
    x, y = G_triang.nodes[node]['pos']
    node_trace_triang['x'] += tuple([x])
    node_trace_triang['y'] += tuple([y])

x_=[]
y_=[]
for n in nodes:
    x0,y0 = G_triang.nodes[n]['pos']
    x_.append(x0)
    y_.append(y0)
    
x, y = G_bar.nodes[Node_]['pos']
node_trace_triang['x'] += tuple([x])
node_trace_triang['y'] += tuple([y])

annotations_=[dict(
                    #
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ),
                        dict(
                        x=x_[0],
                        y=y_[0],
                        xref='x',
                        yref='y',
                        text='ver',
                        showarrow=True,
                        arrowhead=7,
                        ax=0,
                        ay=-20),
                        dict(
                        x=x_[1],
                        y=y_[1],
                        xref='x',
                        yref='y',
                        text='ver',
                        showarrow=True,
                        arrowhead=7,
                        ax=0,
                        ay=-20),
                        dict(
                        x=x_[2],
                        y=y_[2],
                        xref='x',
                        yref='y',
                        text='ver',
                        showarrow=True,
                        arrowhead=7,
                        ax=0,
                        ay=-20),
                        dict(
                        x=x,
                        y=y,
                        xref='x',
                        yref='y',
                        text='bar',
                        showarrow=True,
                        arrowhead=7,
                        ax=0,
                        ay=-40,
                        bgcolor='yellow'),
                        ]
for i in neig_triang.keys():
    annotations_.append(neig_triang[i])
        
for i in range(3):
    annotations_.append(dict(
                        x=x_[i],
                        y=y_[i],
                        xref='x',
                        yref='y',
                        text='ver',
                        showarrow=True,
                        arrowhead=7,
                        ax=0,
                        ay=0,
                        bgcolor='blue'))



fig6 = go.Figure(data=[node_trace_bar_weights,edge_trace_triang,],
             layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                
                titlefont=dict(size=16),
                #showlegend=True,
                hovermode='closest',
                 autosize=False,
                width=900,
                height=800,
                margin=dict(b=20,l=10,r=5,t=40),
                annotations=annotations_,
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))

iplot(fig6, filename='networkx')


# PlotlyVisualizationModule 3: *Pre-extracted graphs*.

In [59]:
#weights=[entry[1] for entry in G_bar.nodes(data='weight')]
max_ = max(weights)

In [63]:
min_=.4 # threshold to build the edges
graph_type='1' #type of graph; if omitted, then graph_type="1" is assumed.
weighting_method='ER' #weighting methods for the edges; if omitted, then weighting_method='ER' is assumed.

G_pre_extracted_1 = pre_extraction.pre_extr(coord, 
                                            topol, 
                                            weights,
                                            triang_source_indices,
                                            triang_sink_indices, 
                                            graph_type = graph_type,
                                            min_= min_)

Gbar 319


In [64]:
G_pre_extracted_1.nodes(data=True)

NodeDataView({0: {'pos': array([0.66446749, 0.943245  ]), 'tdens': 0.42252350313275666, 'terminal': 1}, 1: {'pos': array([0.04201205, 0.13473377]), 'tdens': 0.5559308586597479, 'terminal': -1}, 2: {'pos': array([0.11032514, 0.7616666 ]), 'tdens': 0.6424906481249781, 'terminal': 0}, 3: {'pos': array([0.10128307, 0.60809089]), 'tdens': 0.7530955860231571, 'terminal': 0}, 5: {'pos': array([0.30098159, 0.22949273]), 'tdens': 0.5877263176077935, 'terminal': 0}, 6: {'pos': array([0.39992339, 0.29469301]), 'tdens': 0.9087673602981571, 'terminal': 0}, 8: {'pos': array([0.18446857, 0.55487341]), 'tdens': 0.6931698250577619, 'terminal': 0}, 9: {'pos': array([0.30042281, 0.48513511]), 'tdens': 0.4003091689560735, 'terminal': 0}, 11: {'pos': array([0.13837006, 0.10932997]), 'tdens': 0.4316167481685411, 'terminal': 0}, 12: {'pos': array([0.29520558, 0.39341976]), 'tdens': 0.9079703056699686, 'terminal': 0}, 15: {'pos': array([0.39012743, 0.53672178]), 'tdens': 0.6869519309194618, 'terminal': 0}, 17

In [65]:
edge_trace_filtered_full= go.Scatter(
    x=[],
    y=[],
    line=dict(width=2,color='black'),
    hoverinfo='none',
    mode='lines')

for edge in G_pre_extracted_1.edges():
    x0, y0 = G_pre_extracted_1.nodes[edge[0]]['pos']
    x1, y1 = G_pre_extracted_1.nodes[edge[1]]['pos']
    edge_trace_filtered_full['x'] += tuple([x0, x1, None])
    edge_trace_filtered_full['y'] += tuple([y0, y1, None])


node_trace_bar_weights= go.Scatter(
    x=[],
    y=[],
    text=weights,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=False,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color='orange',#<---------------------- new!
        size=20,
        colorbar=dict(
            thickness=5,
            title='Weights (tdens)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))
    

    
for node in G_pre_extracted_1.nodes():
    if G_pre_extracted_1.nodes[node]['tdens']>min_*0:
        x, y = G_pre_extracted_1.nodes[node]['pos']
        node_trace_bar_weights['x'] += tuple([x])
        node_trace_bar_weights['y'] += tuple([y])


fig7 = go.Figure(data=[
                       edge_trace_filtered_full,node_trace_bar_weights,edge_trace_triang],
                 layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                title='<br>Pre-extracted graph',
                titlefont=dict(size=16),
                showlegend=True,
                hovermode='closest',
                autosize=False,
                width=900,
                height=900,
                margin=dict(b=20,l=10,r=5,t=40),
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))
                 
iplot(fig7, filename='networkx')

In [66]:
min_=.4 # threshold to build the edges
graph_type='2' #type of graph; if omitted, then graph_type="1" is assumed.

G_pre_extracted_2 = pre_extraction.pre_extr(coord, 
                                            topol, 
                                            weights,
                                            triang_source_indices,
                                            triang_sink_indices,
                                            graph_type = graph_type,
                                            min_= min_)

Gbar 319


In [67]:
edge_trace_filtered_full= go.Scatter(
    x=[],
    y=[],
    line=dict(width=3,color='black'),
    hoverinfo='none',
    mode='lines')

for edge in G_pre_extracted_2.edges():
    x0, y0 = G_pre_extracted_2.nodes[edge[0]]['pos']
    x1, y1 = G_pre_extracted_2.nodes[edge[1]]['pos']
    edge_trace_filtered_full['x'] += tuple([x0, x1, None])
    edge_trace_filtered_full['y'] += tuple([y0, y1, None])
    
node_trace_bar_weights= go.Scatter(
    x=[],
    y=[],
    text=weights,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=False,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color='orange',#<---------------------- new!
        size=20,
        colorbar=dict(
            thickness=5,
            title='Weights (tdens)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))
    

    
for node in G_pre_extracted_2.nodes():
    if G_pre_extracted_2.nodes[node]['tdens']>min_*0:
        x, y = G_pre_extracted_2.nodes[node]['pos']
        node_trace_bar_weights['x'] += tuple([x])
        node_trace_bar_weights['y'] += tuple([y])

fig7 = go.Figure(data=[
                       edge_trace_filtered_full,node_trace_bar_weights,edge_trace_triang],
                 layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                title='<br>Pre-extracted graph',
                titlefont=dict(size=16),
                showlegend=True,
                hovermode='closest',
                autosize=False,
                width=900,
                height=900,
                margin=dict(b=20,l=10,r=5,t=40),
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))
                 
iplot(fig7, filename='networkx')

In [72]:
min_=.6 # threshold to build the edges
graph_type='3' #type of graph; if omitted, then graph_type="1" is assumed.

G_pre_extracted_3 = pre_extraction.pre_extr(coord, 
                                            topol, 
                                            weights,
                                            triang_source_indices,
                                            triang_sink_indices,
                                            graph_type = graph_type,
                                            weighting_method='AVG',
                                            min_= min_)

Gbar 319


In [73]:
edge_trace_filtered_full= go.Scatter(
    x=[],
    y=[],
    line=dict(width=3,color='black'),
    hoverinfo='none',
    mode='lines')

for edge in G_pre_extracted_3.edges():
    x0, y0 = G_pre_extracted_3.nodes[edge[0]]['pos']
    x1, y1 = G_pre_extracted_3.nodes[edge[1]]['pos']
    edge_trace_filtered_full['x'] += tuple([x0, x1, None])
    edge_trace_filtered_full['y'] += tuple([y0, y1, None])

node_trace_bar_weights= go.Scatter(
    x=[],
    y=[],
    text=weights,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=False,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color='orange',#<---------------------- new!
        size=20,
        colorbar=dict(
            thickness=5,
            title='Weights (tdens)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))
    

    
for node in G_pre_extracted_3.nodes():
    #if G_pre_extracted_3.nodes[node]['weight']>min_*0:
    x, y = G_pre_extracted_3.nodes[node]['pos']
    node_trace_bar_weights['x'] += tuple([x])
    node_trace_bar_weights['y'] += tuple([y])

fig7 = go.Figure(data=[
                       edge_trace_filtered_full,node_trace_bar_weights,edge_trace_triang],
                 layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
                title='<br>Pre-extracted graph',
                titlefont=dict(size=16),
                showlegend=True,
                hovermode='closest',
                autosize=False,
                width=900,
                height=900,
                margin=dict(b=20,l=10,r=5,t=40),
                xaxis=dict(showgrid=True, zeroline=True, showticklabels=True),
                yaxis=dict(showgrid=True, zeroline=True, showticklabels=True)))
                 
iplot(fig7, filename='networkx')